In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Array of climate zones
climate_zones = ["very_cold", "mixed-humid", "mixed-dry", "marine", "hot-humid", "hot-dry", "cold"]

# Base path for CSV files
base_path = '../data/raw/NREL/america-climate-zones/'

# Columns of interest
columns_of_interest = {
    "electric_stove": "out.electricity.range_oven.energy_consumption.kwh",
    "solar": "out.electricity.pv.energy_consumption.kwh",
    "heat_pump": [
        "out.electricity.heating_hp_bkup.energy_consumption.kwh",
        "out.electricity.heating_hp_bkup_fa.energy_consumption.kwh",
        "out.electricity.cooling.energy_consumption.kwh"
    ]
}

# Function to process and return load profiles
def get_load_profiles(zone):
    # Load the CSV file into a DataFrame
    file_path = f'{base_path}up09-{zone}-single-family_detached.csv'
    df = pd.read_csv(file_path)
    
    # Column names for total and net energy consumption
    total_energy_column = 'out.electricity.total.energy_consumption.kwh'
    net_energy_column = 'out.electricity.net.energy_consumption.kwh'
    scale_factor_column = 'units_represented'
    
    # Ensure units_represented is not zero to avoid division by zero
    df[scale_factor_column].replace(0, 1, inplace=True)
    
    # Convert columns to numeric to avoid TypeErrors during calculations
    df[total_energy_column] = pd.to_numeric(df[total_energy_column], errors='coerce')
    df[net_energy_column] = pd.to_numeric(df[net_energy_column], errors='coerce')
    df[scale_factor_column] = pd.to_numeric(df[scale_factor_column], errors='coerce')
    
    # Scale the data by the number of units represented (divide to get the average per unit)
    df[total_energy_column] = df[total_energy_column] / df[scale_factor_column]
    df[net_energy_column] = df[net_energy_column] / df[scale_factor_column]
    
    # Calculate day of year and filter for summer and winter days
    df['day_of_year'] = df.index // 24
    summer_days = df[df['day_of_year'].isin(range(152, 245))]  # June 1 to August 31
    winter_days = df[(df['day_of_year'] <= 59) | (df['day_of_year'] >= 335)]  # December 1 to February 28
    
    # Reshape data to get average profiles for summer and winter
    summer_reshaped_total = summer_days[total_energy_column].values.reshape(-1, 24)
    winter_reshaped_total = winter_days[total_energy_column].values.reshape(-1, 24)
    
    summer_reshaped_net = summer_days[net_energy_column].values.reshape(-1, 24)
    winter_reshaped_net = winter_days[net_energy_column].values.reshape(-1, 24)
    
    avg_summer_profile_total = summer_reshaped_total.mean(axis=0)
    avg_winter_profile_total = winter_reshaped_total.mean(axis=0)
    
    avg_summer_profile_net = summer_reshaped_net.mean(axis=0)
    avg_winter_profile_net = winter_reshaped_net.mean(axis=0)
    
    # Initialize dictionary for additional loads
    additional_loads = {
        "electric_stove": None,
        "solar": None,
        "heat_pump": None
    }
    
    # Process columns of interest
    for key, col_name in columns_of_interest.items():
        if isinstance(col_name, list):  # Handle list of columns (heat pump)
            for cn in col_name:
                df[cn] = pd.to_numeric(df[cn], errors='coerce')
                df[cn] = df[cn] / df[scale_factor_column]
            summer_values = [df[cn][summer_days.index].values.reshape(-1, 24).mean(axis=0) for cn in col_name]
            winter_values = [df[cn][winter_days.index].values.reshape(-1, 24).mean(axis=0) for cn in col_name]
            additional_loads[key] = {
                "summer": np.sum(summer_values, axis=0).tolist(),
                "winter": np.sum(winter_values, axis=0).tolist()
            }
        else:
            df[col_name] = pd.to_numeric(df[col_name], errors='coerce')
            df[col_name] = df[col_name] / df[scale_factor_column]
            summer_values = df[col_name][summer_days.index].values.reshape(-1, 24).mean(axis=0)
            winter_values = df[col_name][winter_days.index].values.reshape(-1, 24).mean(axis=0)
            additional_loads[key] = {
                "summer": summer_values.tolist(),
                "winter": winter_values.tolist()
            }
    
    # Plotting
    # plt.figure(figsize=(15, 7))
    
    # # Plot total and net energy consumption for both summer and winter
    # plt.plot(range(1, 25), avg_summer_profile_total, label='Average Summer Load Shape - Total', color='orange')
    # plt.plot(range(1, 25), avg_winter_profile_total, label='Average Winter Load Shape - Total', color='blue')
    # plt.plot(range(1, 25), avg_summer_profile_net, label='Average Summer Load Shape - Net', color='green')
    # plt.plot(range(1, 25), avg_winter_profile_net, label='Average Winter Load Shape - Net', color='red')
    
    # for key, profiles in additional_loads.items():
    #     plt.plot(range(1, 25), profiles['summer'], label=f'Summer Load - {key.replace("_", " ").capitalize()}', linestyle='dotted')
    #     plt.plot(range(1, 25), profiles['winter'], label=f'Winter Load - {key.replace("_", " ").capitalize()}', linestyle='dotted')
    
    # plt.xlabel('Hour of the Day')
    # plt.ylabel('Average Energy Consumption per Unit (kWh)')
    # plt.title(f'{zone.capitalize()} - Average Daily Load Shapes for Summer and Winter')
    # plt.legend()
    # plt.grid(True)
    # plt.tight_layout()
    # plt.show()

    return {
        "summer": {
            "total": avg_summer_profile_total.tolist(),
            "net": avg_summer_profile_net.tolist(),
            **{key: value["summer"] for key, value in additional_loads.items()}
        },
        "winter": {
            "total": avg_winter_profile_total.tolist(),
            "net": avg_winter_profile_net.tolist(),
            **{key: value["winter"] for key, value in additional_loads.items()}
        }
    }

# Iterate through each climate zone and process
load_profiles = {}
for zone in climate_zones:
    load_profiles[zone] = get_load_profiles(zone)

# Example usage: Print the load profiles for "very_cold"
print(load_profiles["very_cold"])


/var/folders/gr/g6k4s8t13n98c8bw4ywdpxrr0000gn/T/ipykernel_17094/744257241.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[scale_factor_column].replace(0, 1, inplace=True)
/var/folders/gr/g6k4s8t13n98c8bw4ywdpxrr0000gn/T/ipykernel_17094/744257241.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values a

{'summer': {'total': [0.6630204343077961, 0.589933725758449, 0.5526774853590631, 0.5451396649385561, 0.5605857424875195, 0.6224244131624428, 0.7385491311443937, 0.8286891141033031, 0.8776349786386329, 0.9201667956749234, 0.9585293868807612, 0.9909558671754999, 1.0240784760224657, 1.0432682831701234, 1.0798957913306453, 1.155917329709102, 1.274332628288211, 1.379137996831798, 1.409459065380185, 1.3705706095190098, 1.3110659472206232, 1.2275093786002313, 1.0210932819700465, 0.7962222302227345], 'net': [0.6630204343077961, 0.589933725758449, 0.5526774853590631, 0.5451396649385561, 0.5605857424875195, 0.6224244131624428, 0.738546229958718, 0.8286174965197777, 0.8774084521409373, 0.9197987621207764, 0.9580551915322586, 0.9904135014640943, 1.023515303979455, 1.0427244263632887, 1.0793892809139791, 1.1554798207085264, 1.2739967177899392, 1.378906117991552, 1.4093413228446627, 1.3705275657642098, 1.3110503882248465, 1.2275088055635566, 1.0210932819700465, 0.7962222302227345], 'electric_stove':

/var/folders/gr/g6k4s8t13n98c8bw4ywdpxrr0000gn/T/ipykernel_17094/744257241.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[scale_factor_column].replace(0, 1, inplace=True)


In [2]:
for zone in load_profiles:
    print(zone)
    
    # Calculate the resulting summer load profile
    summer_net = np.array(load_profiles[zone]["summer"]["net"])
    summer_electric_stove = summer_net + np.array(load_profiles[zone]["summer"]["electric_stove"])
    summer_solar = np.array(load_profiles[zone]["summer"]["solar"])
    summer_heat_pump = summer_net + np.array(load_profiles[zone]["summer"]["heat_pump"])
    # summer_resulting_load = summer_net + summer_electric_stove + summer_solar + summer_heat_pump
    
    # print("Summer Net Load Profile:")
    # print(load_profiles[zone]["summer"]["net"])
    print("Summer Heat Pump:")
    print(summer_heat_pump.tolist())
    
    # Calculate the resulting winter load profile
    winter_net = np.array(load_profiles[zone]["winter"]["net"])
    winter_electric_stove = winter_net + np.array(load_profiles[zone]["winter"]["electric_stove"])
    winter_solar = np.array(load_profiles[zone]["winter"]["solar"])
    winter_heat_pump = winter_net + np.array(load_profiles[zone]["winter"]["heat_pump"])
    # winter_resulting_load = winter_net + winter_electric_stove + winter_solar + winter_heat_pump
    
    # print("Winter Net Load Profile:")
    # print(load_profiles[zone]["winter"]["net"])
    print("Winter Heat Pump")
    print(winter_heat_pump.tolist())
    # print("Winter Resulting Load Profile (Net + Additions):")
    # print(winter_resulting_load.tolist())
    
    print()


very_cold
Summer Heat Pump:
[0.8169385170650927, 0.7211163714477733, 0.6636717279905916, 0.6418396037346392, 0.64062832421275, 0.6916853848646318, 0.8090575796851005, 0.9139277133736563, 0.9876557999711986, 1.0613178763440867, 1.1357439006096397, 1.2087386742751545, 1.284879551291283, 1.3490355672763075, 1.4290771049347166, 1.549654953917052, 1.704674854190669, 1.828984681019586, 1.8518346414170517, 1.7712332289266524, 1.6475487351190479, 1.5021473664314522, 1.2463262138776887, 0.980258709557412]
Winter Heat Pump
[3.240597925967263, 3.1271204520089295, 3.080324020337302, 3.0962771887400806, 3.1121975942460325, 3.201513371155756, 3.3415681175595244, 3.531802883184525, 3.666544190228175, 3.6869706132192506, 3.6616101376488093, 3.5812653800843264, 3.50202543092758, 3.4146854042658754, 3.262323332093256, 3.171707948908733, 3.183696862599209, 3.32752941778274, 3.49964406622024, 3.634282936507939, 3.6724622550843273, 3.6589358630952407, 3.5743544084821437, 3.424745182291668]

mixed-humid
Sum